<a href="https://colab.research.google.com/github/GoldenWanGila/Collage-Project/blob/master/DEF%E8%87%AA%E5%BC%B7%E5%8D%97%E9%83%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
from re import T
import sklearn.linear_model
import seaborn as sns; sns.set()
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sympy import *
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import matplotlib as mpl
import pandas as pd
import numpy as np
import datetime

In [37]:
def plot(x:np.ndarray, y:np.ndarray):
    x = x.reshape(-1,1)
    xgbrModel=xgb.XGBRegressor()
    xgbrModel.fit(x,y)
    predicted=xgbrModel.predict(x)
    v = mean_squared_error(y,predicted)
    print(sqrt(v))

In [38]:
def arrangeDataFrame(x_dataFrame:pd.DataFrame,y_dataFrame:pd.DataFrame,direction:int):
    x_dataFrame_direction = x_dataFrame[x_dataFrame['direction']==direction]
    y_dataFrame_direction = y_dataFrame[y_dataFrame['direction']==direction]

    if len(set(x_dataFrame_direction['date'])) != len(set(y_dataFrame_direction['date'])):
        same_dates = [date for date in set(y_dataFrame_direction['date']) if (date in set(x_dataFrame_direction['date']))]
        x_dataFrame_direction = x_dataFrame_direction[x_dataFrame_direction['date'].isin(same_dates)]
        y_dataFrame_direction = y_dataFrame_direction[y_dataFrame_direction['date'].isin(same_dates)]

    same_nums = [num for num in set(y_dataFrame_direction['num']) if (num in set(x_dataFrame_direction['num']))]
    x_target_dataFrame = x_dataFrame_direction[x_dataFrame_direction['num'].isin(same_nums)]
    x_target_dataFrame.sort_values(by=['date','num'], inplace=True)
    x_target_dataFrame.reset_index(inplace=True)
    y_target_dataFrame = y_dataFrame_direction[y_dataFrame_direction['num'].isin(same_nums)]
    y_target_dataFrame.sort_values(by=['date','num'], inplace=True)
    y_target_dataFrame.reset_index(inplace=True)

    # 獲取 delay不存在的 row index
    null_index = list(x_target_dataFrame[x_target_dataFrame['delay'].isnull()].index)
    null_index.extend(list(y_target_dataFrame[y_target_dataFrame['delay'].isnull()].index))
    null_index = set(null_index)

    # 將delay不存在的 row 從資料中移除
    x_target_dataFrame = x_target_dataFrame.drop(index=null_index)
    y_target_dataFrame = y_target_dataFrame.drop(index=null_index)

    x = x_target_dataFrame['delay'].to_numpy()
    y = y_target_dataFrame['delay'].to_numpy()
    return x,y

In [39]:
def getRMSE(x:np.ndarray, y:np.ndarray):
    x = x.reshape(-1,1)
    xgbrModel=xgb.XGBRegressor()
    xgbrModel.fit(x,y)
    predicted=xgbrModel.predict(x)
    v = mean_squared_error(y,predicted)
    return sqrt(v)

In [40]:
dataFrame = pd.read_csv('/content/drive/MyDrive/railway.csv')
label_map = {'Keelung':1,'Xizhi':2,'Taipei':3,'Banqiao':4,'Taoyuan':5,'Zhongli':6,'Hsinchu':7,'Zhunan':8,'Miaoli':9,'Fengyuan':10,'Taichung':11,'Changhua':12,'Yuanlin':13,
'Douliu':14,'Chiayi':15,'Tainan':16,'Xinzuoying':17,'Kaohsiung':18,'Tanzi':0,'Taiyuan':0,'Xinwuri':0}

In [41]:
dataFrame_localTrain = dataFrame[dataFrame['num']<500]
dataFrame_localTrain.sort_values(by=["time","date"],inplace=True)
dataFrame_localTrain = dataFrame_localTrain[dataFrame_localTrain["date"]>='2021/01/22']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


**連假日**

In [42]:
listlist = ["2021/02/10","2021/02/11","2021/02/12","2021/02/13","2021/02/14","2021/02/15","2021/02/16","2021/04/02","2021/04/03","2021/04/04","2021/04/05","2021/02/26","2021/02/27","2021/02/28","2021/03/01"]

In [43]:
dataFrame_holiday = dataFrame_localTrain[dataFrame_localTrain["date"].isin(listlist)]

In [44]:

dataFrame_holiday_Taichung = dataFrame_holiday[dataFrame_holiday['Station']=='Taichung']
dataFrame_holiday_Changhua = dataFrame_holiday[dataFrame_holiday['Station']=='Changhua']
dataFrame_holiday_Yuanlin = dataFrame_holiday[dataFrame_holiday['Station']=='Yuanlin']
dataFrame_holiday_Douliu = dataFrame_holiday[dataFrame_holiday['Station']=='Douliu']
dataFrame_holiday_Chiayi = dataFrame_holiday[dataFrame_holiday['Station']=="Chiayi"]
dataFrame_holiday_Tainan = dataFrame_holiday[dataFrame_holiday['Station']=='Tainan']
dataFrame_holiday_Xinzuoying = dataFrame_holiday[dataFrame_holiday['Station']=='Xinzuoying']
dataFrame_holiday_Kaohsiung = dataFrame_holiday[dataFrame_holiday['Station']=='Kaohsiung']

In [45]:
dataFrameList = [dataFrame_holiday_Taichung, dataFrame_holiday_Changhua, dataFrame_holiday_Yuanlin, dataFrame_holiday_Douliu, dataFrame_holiday_Chiayi, dataFrame_holiday_Tainan, 
                 dataFrame_holiday_Xinzuoying,dataFrame_holiday_Kaohsiung]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')
stationList = ['Taichung','Changhua','Yuanlin','Douliu','Chiayi','Tainan','Xinzuoying','Kaohsiung']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

...


[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [46]:
mse_df.to_csv("南部連假日MSE.csv")
mse_df

,Taichung,Changhua,Yuanlin,Douliu,Chiayi,Tainan,Xinzuoying,Kaohsiung
Taichung,--,1.07239469985279,1.39164894935056,1.62796880105757,2.00106527524437,3.86344675887665,3.08004381342069,3.16476965725521
Changhua,1.07402604019298,--,0.853565332161804,1.44998199874444,1.94862646919907,2.39410459735512,3.09299293429897,3.22995995576462
Yuanlin,1.23332004255336,0.904443105968481,--,1.18376354473664,1.75763230925379,2.30620684727409,3.00424171807553,3.20256057611261
Douliu,1.56145958277216,1.27490229254549,0.996035906849632,--,1.31822309462975,1.90135296216830,2.82141991293265,3.04380095806218
Chiayi,1.77430000577339,1.54369893964938,1.38198249647286,0.968794972359004,--,3.56347616866422,2.64062024812199,2.93074468368303
Tainan,2.89997460927953,2.93291749137497,2.92104131263696,2.82358743943271,2.41821605093809,--,2.14996004999179,3.49565104397429
Xinzuoying,3.20878495827396,3.30889752539582,3.31803489665828,3.18240952737996,2.76356179232135,1.21786039160456,--,1.23993949342171
Kaohsiung,3.00455215123243,3.26675050615567,3.32439110385158,3.22481601400141,2.76264426743984,1.45072428421369,0.896290572392181,--


**全部**

In [47]:
dataFrame_localTrain = dataFrame_localTrain[dataFrame_localTrain["date"]!="2021/04/02"]
dataFrame_localTrain = dataFrame_localTrain[dataFrame_localTrain["date"]!="2021/04/03"]

In [48]:
dataFrame_localTrain_Taichung = dataFrame_localTrain[dataFrame_localTrain['Station']=='Taichung']
dataFrame_localTrain_Changhua = dataFrame_localTrain[dataFrame_localTrain['Station']=='Changhua']
dataFrame_localTrain_Yuanlin = dataFrame_localTrain[dataFrame_localTrain['Station']=='Yuanlin']
dataFrame_localTrain_Douliu = dataFrame_localTrain[dataFrame_localTrain['Station']=='Douliu']
dataFrame_localTrain_Chiayi = dataFrame_localTrain[dataFrame_localTrain['Station']=="Chiayi"]
dataFrame_localTrain_Tainan = dataFrame_localTrain[dataFrame_localTrain['Station']=='Tainan']
dataFrame_localTrain_Xinzuoying = dataFrame_localTrain[dataFrame_localTrain['Station']=='Xinzuoying']
dataFrame_localTrain_Kaohsiung = dataFrame_localTrain[dataFrame_localTrain['Station']=='Kaohsiung']


In [49]:
dataFrameList = [dataFrame_localTrain_Taichung, dataFrame_localTrain_Changhua, dataFrame_localTrain_Yuanlin, dataFrame_localTrain_Douliu, dataFrame_localTrain_Chiayi, dataFrame_localTrain_Tainan, 
                 dataFrame_localTrain_Xinzuoying,dataFrame_localTrain_Kaohsiung]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')
stationList = ['Taichung','Changhua','Yuanlin','Douliu','Chiayi','Tainan','Xinzuoying','Kaohsiung']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)
mse_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/i

...

[12:33:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[12:33:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Taichung,Changhua,Yuanlin,Douliu,Chiayi,Tainan,Xinzuoying,Kaohsiung
Taichung,--,1.00341435449175,1.47706836191421,1.72082876772287,1.86072341942092,2.26231297503072,2.69724205974136,2.68683312479476
Changhua,1.02191160927636,--,1.03353399489176,1.52300534279945,1.75844690853894,2.30854674535143,2.68366010589180,2.82004455144571
Yuanlin,1.23985138306045,0.857529282218033,--,1.22413899385033,1.63834913526334,2.17342967320874,2.65949074468700,2.82351062528594
Douliu,1.51918063400115,1.22939047018024,1.07063877574045,--,1.08354140557568,1.80613840433037,2.42334084838322,2.59413685305660
Chiayi,1.78316959386850,1.55355802623673,1.50821248291134,1.10795842493284,--,1.78835690843774,2.35373170394649,2.53108426389672
Tainan,2.41238935856458,2.32591952701402,2.39178658438311,2.17732149232505,1.74973431712536,--,1.96214463873306,2.36847820644942
Xinzuoying,2.69092957184025,2.66076565267503,2.80163184632108,2.58781308219706,2.17887986282639,1.44509839135383,--,1.21141040461352
Kaohsiung,2.70175266150222,2.74764207635499,2.90228734666299,2.69500109611355,2.24969593484156,1.59471813011664,0.902805792726486,--


In [50]:
dataFrame_localTrain_weekday = dataFrame_localTrain[dataFrame_localTrain['date'].apply(lambda date: datetime.date(int(date[:4]), int(date[5:7]), int(date[8:])).isoweekday() < 6)]
dataFrame_localTrain_weekday

,Station,direction,id,date,time,num,dest,delay,type
242276,Taipei,0,1,2021/01/22,00:02,251,Shulin,4.0,B
242748,Taipei,0,1,2021/01/25,00:02,251,Shulin,3.0,B
242904,Taipei,0,1,2021/01/26,00:02,251,Shulin,0.0,B
243056,Taipei,0,1,2021/01/27,00:02,251,Shulin,1.0,B
243208,Taipei,0,1,2021/01/28,00:02,251,Shulin,0.0,B
...,...,...,...,...,...,...,...,...,...
312912,Kaohsiung,0,96,2021/03/30,23:55,149,Chaozhou,16.0,B
313007,Kaohsiung,0,95,2021/03/31,23:55,149,Chaozhou,0.0,B
313102,Kaohsiung,0,95,2021/04/01,23:55,149,Chaozhou,0.0,B
313486,Kaohsiung,0,95,2021/04/05,23:55,149,Chaozhou,21.0,B


In [51]:
dataFrame_localTrain_weekend = dataFrame_localTrain[dataFrame_localTrain['date'].apply(lambda date: datetime.date(int(date[:4]), int(date[5:7]), int(date[8:])).isoweekday() > 5)]
dataFrame_localTrain_weekend

,Station,direction,id,date,time,num,dest,delay,type
242433,Taipei,0,1,2021/01/23,00:02,251,Shulin,5.0,B
242587,Taipei,0,1,2021/01/24,00:02,249,Shulin,3.0,B
243376,Taipei,0,1,2021/01/30,00:02,251,Shulin,0.0,B
243515,Taipei,0,1,2021/01/31,00:02,249,Shulin,6.0,B
244585,Taipei,0,1,2021/02/06,00:02,251,Shulin,0.0,B
...,...,...,...,...,...,...,...,...,...
311954,Kaohsiung,0,95,2021/03/20,23:55,149,Chaozhou,2.0,B
312051,Kaohsiung,0,97,2021/03/21,23:55,149,Chaozhou,0.0,B
312623,Kaohsiung,0,95,2021/03/27,23:55,149,Chaozhou,0.0,B
312720,Kaohsiung,0,97,2021/03/28,23:55,149,Chaozhou,3.0,B


**假日**

In [52]:
dataFrame_localTrain_weekend_Taichung = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Taichung']
dataFrame_localTrain_weekend_Changhua = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Changhua']
dataFrame_localTrain_weekend_Yuanlin = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Yuanlin']
dataFrame_localTrain_weekend_Douliu = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Douliu']
dataFrame_localTrain_weekend_Chiayi = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=="Chiayi"]
dataFrame_localTrain_weekend_Tainan = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Tainan']
dataFrame_localTrain_weekend_Xinzuoying = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Xinzuoying']
dataFrame_localTrain_weekend_Kaohsiung = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Kaohsiung']

In [53]:
dataFrameList = [dataFrame_localTrain_weekend_Taichung,dataFrame_localTrain_weekend_Changhua,dataFrame_localTrain_weekend_Yuanlin,dataFrame_localTrain_weekend_Douliu,dataFrame_localTrain_weekend_Chiayi,dataFrame_localTrain_weekend_Tainan,
                 dataFrame_localTrain_weekend_Xinzuoying,dataFrame_localTrain_weekend_Kaohsiung]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')

stationList = ['Taichung','Changhua','Yuanlin','Douliu','Chiayi','Tainan','Xinzuoying','Kaohsiung']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [54]:
mse_df.to_csv("南部假日MSE.csv")
mse_df

,Taichung,Changhua,Yuanlin,Douliu,Chiayi,Tainan,Xinzuoying,Kaohsiung
Taichung,--,0.979091097994795,1.24058279139674,1.49890528548391,1.87170294496877,2.20178648206410,3.20692826545473,3.18868097694495
Changhua,1.14582554677509,--,0.691786295532000,1.28300007906644,1.71899006464995,2.18413458171505,3.05460145943530,3.24662338978319
Yuanlin,1.30350209817521,0.833991229149296,--,1.13941623987025,1.62808223819458,2.11231070978449,3.00546779458788,3.19288315698133
Douliu,1.65261393814660,1.24785573055632,1.09492166190455,--,1.19673708086683,1.75917414666289,2.81971251213465,2.98895134627932
Chiayi,1.88190697563965,1.52358452992595,1.53953528377657,1.09649729253559,--,1.78683299240139,2.68044661991837,2.85643185507261
Tainan,2.13948159613989,1.86901052949217,1.87011463083388,1.59947997519265,1.07481800808785,--,2.31486929151088,2.66404938867003
Xinzuoying,2.30817620518935,2.06793982293398,2.16578586822207,1.88967785985104,1.36515300765757,1.17703941365718,--,1.15279199428197
Kaohsiung,2.47796138722390,2.24633618933239,2.37836395745664,2.09222250689882,1.60830111658873,1.43901771258048,0.970622711098872,--


**平日**

In [55]:
dataFrame_localTrain_weekday_Taichung = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Taichung']
dataFrame_localTrain_weekday_Changhua = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Changhua']
dataFrame_localTrain_weekday_Yuanlin = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Yuanlin']
dataFrame_localTrain_weekday_Douliu = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Douliu']
dataFrame_localTrain_weekday_Chiayi = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=="Chiayi"]
dataFrame_localTrain_weekday_Tainan = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Tainan']
dataFrame_localTrain_weekday_Xinzuoying = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Xinzuoying']
dataFrame_localTrain_weekday_Kaohsiung = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Kaohsiung']

In [56]:
dataFrameList = [dataFrame_localTrain_weekday_Taichung,dataFrame_localTrain_weekday_Changhua,dataFrame_localTrain_weekday_Yuanlin,dataFrame_localTrain_weekday_Douliu,dataFrame_localTrain_weekday_Chiayi,dataFrame_localTrain_weekday_Tainan,
                 dataFrame_localTrain_weekday_Xinzuoying,dataFrame_localTrain_weekday_Kaohsiung]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')

stationList = ['Taichung','Changhua','Yuanlin','Douliu','Chiayi','Tainan','Xinzuoying','Kaohsiung']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

...

[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:33:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [57]:
mse_df.to_csv("南部平日MSE.csv")
mse_df

,Taichung,Changhua,Yuanlin,Douliu,Chiayi,Tainan,Xinzuoying,Kaohsiung
Taichung,--,0.979272615978290,1.24248368863747,1.57332621109798,1.72746271584365,2.09204704324653,2.33672368537901,2.39167712231963
Changhua,0.946945758721596,--,0.783582800425280,1.39563903375047,1.70253711331916,2.19328295149636,2.43770835227718,2.57619890550534
Yuanlin,1.17437283952982,0.857453368679126,--,1.24064880359830,1.62254794583243,2.15819608625580,2.43675422841944,2.63779489957842
Douliu,1.41592849050631,1.20893936174121,1.04997536127267,--,1.01793578762907,1.78147584069363,2.17241930789180,2.39353817628444
Chiayi,1.70217854563776,1.54260453650300,1.48002820251773,1.10637848504491,--,1.76682827042019,2.14977298042224,2.36987952349296
Tainan,2.45487962517257,2.46627962462711,2.55589822360360,2.35904454574594,1.95313415592231,--,1.71114550555347,2.21661828241988
Xinzuoying,2.78156934005675,2.82201967172031,2.97978831567054,2.78682833454473,2.41028409335685,1.52219168736917,--,1.22440786568002
Kaohsiung,2.74063232629186,2.89023390891853,3.06280678903833,2.88460237908409,2.44544975967688,1.63301464592401,0.864262422280268,--
